# Intento con Pdfminer.six

In [ ]:
from pdfminer.high_level import extract_text
from pypdf import PdfReader, PdfWriter
import fitz

remove_pages={
    "Kasarakuy raymimanta": (6, 7)
}

In [ ]:
key="Kasarakuy raymimanta"


reader = PdfReader(f'data/{key}.pdf')
number_of_pages = len(reader.pages)

list_pages=range(remove_pages[key][0], number_of_pages-remove_pages[key][1])
extracted=extract_text("data/{}.pdf".format(key), page_numbers=list_pages)

In [49]:
import re

doc = fitz.open(f'data/{key}.pdf')

i=remove_pages[key][0]
full_text=""
for page in doc.pages(remove_pages[key][0], number_of_pages-remove_pages[key][1]):
    # print("Page number: ", i)
    this_page_text=page.get_text("text", sort=True).replace("\n", " ")
    this_page_text=re.sub(r'\d+', '', this_page_text)
    full_text+=this_page_text
    i+=1

In [53]:
from nltk.tokenize import sent_tokenize, word_tokenize
from langdetect import detect

sentences = sent_tokenize(full_text)
non_spanish_sentences = []

#Oraciones antes de filtrar
print("Oraciones antes de filtrar: ", len(sentences))
for sentence in sentences:
    try:
        if detect(sentence) != 'es':
            non_spanish_sentences.append(sentence)
    except:
        pass

sentences = non_spanish_sentences
print("Oraciones después de filtrar: ", len(sentences))

Oraciones antes de filtrar:  1115
Oraciones después de filtrar:  533


In [55]:
#a, aa, ch, chh, ch', ts, tr, h, i, ii, k, kh, k', l, ll, m, n, ñ, p, ph, p', q, qh, q', r, s, sh, t, th, t', u, uu, w, y
alfabeto_quechua = ['a', 'aa', 'ch', 'chh', 'ch\'', 'ts', 'tr', 'h', 'i', 'ii', 'k', 'kh', 'k\'', 'l', 'll', 'm', 'n', 'ñ', 'p', 'ph', 'p\'', 'q', 'qh', 'q\'', 'r', 's', 'sh', 't', 'th', 't\'', 'u', 'uu', 'w', 'y']

def rule_based_heuristic(sentence):
    #exclude sentences containing words formed by graphemes outside the alphabet
    words = word_tokenize(sentence)
    for word in words:
        if word not in alfabeto_quechua:
            return False
        
    return True

# Intento fallido con pypdf

In [ ]:
from pypdf import PdfReader, PdfWriter

In [ ]:
#Las páginas iniciales y finales de los documentos que se quieren eliminar
remove_pages={
    "Kasarakuy raymimanta": (6, 7)
}

In [ ]:

for key in remove_pages:
    reader = PdfReader(f'data/{key}.pdf')
    number_of_pages = len(reader.pages)
    print(f'Number of pages: {number_of_pages}')

    #Eliminar las páginas iniciales y finales
    writer = PdfWriter()

    for page in reader.pages[remove_pages[key][0]:-remove_pages[key][1]]:
        writer.add_page(page)


In [ ]:
page=writer.pages[0]

In [ ]:
def visitor_body(text, cm, tm, font_dict, font_size):
    curr_font_size = font_size
    print(f'{text} {curr_font_size} {font_size}')

In [ ]:
page.extract_text()